In [1]:
import pandas as pd
import sqlite3 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
plt.style.use('fivethirtyeight')
conn = sqlite3.connect("data/dataset.sqlite")
dataset = "'dataset_2012-24_new'"


df = pd.read_sql_query(f"SELECT * from {dataset}", conn)

list(df.columns)

['index',
 'TEAM_NAME',
 'GP',
 'W',
 'L',
 'W_PCT',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'FGM_RANK',
 'FGA_RANK',
 'FG_PCT_RANK',
 'FG3M_RANK',
 'FG3A_RANK',
 'FG3_PCT_RANK',
 'FTM_RANK',
 'FTA_RANK',
 'FT_PCT_RANK',
 'OREB_RANK',
 'DREB_RANK',
 'REB_RANK',
 'AST_RANK',
 'TOV_RANK',
 'STL_RANK',
 'BLK_RANK',
 'BLKA_RANK',
 'PF_RANK',
 'PFD_RANK',
 'PTS_RANK',
 'PLUS_MINUS_RANK',
 'Date',
 'TEAM_NAME.1',
 'GP.1',
 'W.1',
 'L.1',
 'W_PCT.1',
 'MIN.1',
 'FGM.1',
 'FGA.1',
 'FG_PCT.1',
 'FG3M.1',
 'FG3A.1',
 'FG3_PCT.1',
 'FTM.1',
 'FTA.1',
 'FT_PCT.1',
 'OREB.1',
 'DREB.1',
 'REB.1',
 'AST.1',
 'TOV.1',
 'STL.1',
 'BLK.1',
 'BLKA.1',
 'PF.1',
 'PFD.1',
 'PTS.1',
 'PLUS_MINUS.1',
 'GP_RANK.1',
 'W_RANK.1',
 'L_RANK.1',
 'W_PCT_RANK.1',
 'MIN_RANK.1',
 'FGM_RANK.1',
 'FG

# Data Cleaning

In [2]:
# find all time gaps and plot
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df['Time_Gap'] = df['Date'].diff().dt.days

df['Season'] = 0
season = 2012
# Iterate through the DataFrame to set the season based on time gaps
for i in range(len(df)):
    if df['Time_Gap'].iloc[i] > 80:
        df.at[i, 'Season'] = season
        season += 1
    else:
        df.at[i, 'Season'] = season

df = df.drop(columns=['Time_Gap'])
print(len(df))

# rename Days-Rest-Home and Days-Rest-Away to Days-Rest and Days-Rest.1
df = df.rename(columns={'Days-Rest-Home': 'Days-Rest', 'Days-Rest-Away': 'Days-Rest.1'})


df

15115


,index,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,PTS_RANK.1,PLUS_MINUS_RANK.1,Date.1,Score,Home-Team-Win,OU,OU-Cover,Days-Rest,Days-Rest.1,Season
0,0,New York Knicks,1.0,1.0,0.0,1.000,48.0,36.0,84.0,0.429,...,30.0,5.0,2012-11-04,184.0,1.0,188.0,0.0,2.0,4.0,2012
1,1,Toronto Raptors,2.0,0.0,2.0,0.000,48.0,35.0,86.5,0.405,...,25.0,3.0,2012-11-04,191.0,1.0,187.0,1.0,1.0,2.0,2012
2,2,Orlando Magic,1.0,1.0,0.0,1.000,48.0,42.0,86.0,0.488,...,27.0,17.0,2012-11-04,209.0,1.0,194.5,1.0,2.0,2.0,2012
3,3,Oklahoma City Thunder,2.0,1.0,1.0,0.500,48.0,34.0,76.5,0.444,...,6.0,24.0,2012-11-04,199.0,0.0,199.0,2.0,2.0,2.0,2012
4,4,Los Angeles Lakers,3.0,0.0,3.0,0.000,48.0,36.0,72.3,0.498,...,24.0,22.0,2012-11-04,187.0,1.0,192.0,0.0,2.0,2.0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15108,15108,New Orleans Pelicans,82.0,49.0,33.0,0.598,48.1,42.5,87.4,0.486,...,3.0,2.0,2024-04-27,191.0,0.0,210.0,0.0,3.0,3.0,2023
15112,15112,Dallas Mavericks,82.0,50.0,32.0,0.610,48.1,43.1,89.7,0.481,...,12.0,7.0,2024-04-28,227.0,0.0,209.5,1.0,2.0,2.0,2023
15113,15113,Indiana Pacers,82.0,47.0,35.0,0.573,48.1,47.0,92.7,0.507,...,4.0,11.0,2024-04-28,53.0,1.0,214.5,0.0,2.0,2.0,2023
15111,15111,Philadelphia 76ers,82.0,47.0,35.0,0.573,48.2,41.5,89.4,0.464,...,19.0,5.0,2024-04-28,189.0,0.0,209.0,0.0,3.0,3.0,2023


# Feature Engineering

In [3]:
# elo ratings
def initialize_elo(teams, initial_elo=1500):
    return {team: initial_elo for team in teams}

def expected_score(elo1, elo2, home_advantage=0.6):
    return 1 / (1 + 10**(((elo2 - elo1) + home_advantage * 100) / 400))

def update_elo(elo, team1, team2, result, home_advantage=0.6, k=20):
    expected1 = expected_score(elo[team1], elo[team2], home_advantage)
    expected2 = expected_score(elo[team2], elo[team1], -home_advantage)

    if result == 1:  # team1 wins
        elo[team1] += k * (1 - expected1)
        elo[team2] += k * (0 - expected2)
    else:  # team2 wins
        elo[team1] += k * (0 - expected1)
        elo[team2] += k * (1 - expected2)
    
    return elo

def weighted_initial_elo(teams, end_season_elo, initial_elo=1500, weight=0.75):
    return {team: int(end_season_elo.get(team, initial_elo) * weight + initial_elo * (1 - weight)) for team in teams}

def add_elo_scores(data, home_advantage=0.6, initial_elo=1500, weight=0.75):
    teams = pd.concat([data['TEAM_NAME'], data['TEAM_NAME.1']]).unique()
    elo = initialize_elo(teams, initial_elo)
    elos = []
    
    current_season = data.iloc[0]['Season']
    end_season_elo = {}

    for index, row in data.iterrows():
        if row['Season'] != current_season:
            current_season = row['Season']
            elo = weighted_initial_elo(teams, end_season_elo, initial_elo, weight)

        team1 = row['TEAM_NAME']
        team2 = row['TEAM_NAME.1']
        result = row['Home-Team-Win']

        elos.append((elo[team1], elo[team2]))
        
        # Update Elo ratings after the game
        elo = update_elo(elo, team1, team2, result, home_advantage)
        
        # Update end-of-season Elo ratings
        end_season_elo = elo.copy()

    data[['Elo_Team', 'Elo_Team.1']] = pd.DataFrame(elos, index=data.index)
    return data

def verify_elo_scores(data, team):
    elo_cols = ['Elo_Team', 'Elo_Team.1', 'Date', 'TEAM_NAME', 'TEAM_NAME.1', 'Home-Team-Win']
    team_data = data[(data['TEAM_NAME'] == team) | (data['TEAM_NAME.1'] == team)]
    elo_df = team_data[elo_cols]
    
    home_elo = elo_df[['Elo_Team', 'TEAM_NAME', 'Date']].copy()
    home_elo.columns = ['Elo', 'TEAM_NAME', 'Date']
    
    away_elo = elo_df[['Elo_Team.1', 'TEAM_NAME.1', 'Date']].copy()
    away_elo.columns = ['Elo', 'TEAM_NAME', 'Date']
    
    elo_df_long = pd.concat([home_elo, away_elo], axis=0, ignore_index=True)
    elo_df_long = elo_df_long[elo_df_long['TEAM_NAME'] == team]
    elo_df_long = elo_df_long.sort_values(by='Date').set_index('Date')
    
    plt.figure(figsize=(12, 6))
    plt.plot(elo_df_long['Elo'], marker='o', markersize=1, color='b', linewidth=2)
    plt.title(f'Elo ratings for {team}')
    plt.xticks(ticks=elo_df_long.index[::100], rotation=45)
    plt.show()
    
def get_elo_acc(data):
    elo_home = data['Elo_Team']
    elo_away = data['Elo_Team.1']
    elo_pred = (elo_home > elo_away).astype(int)
    print(f"acc {round(accuracy_score(data['Home-Team-Win'], elo_pred), 3)}  f1 {round(f1_score(data['Home-Team-Win'], elo_pred), 3)}")
    

# Assuming df is your DataFrame with match data
data = add_elo_scores(df)
# verify_elo_scores(data, 'Minnesota Timberwolves')
get_elo_acc(data)

data = data.drop(columns=['Score','OU','OU-Cover'])


acc 0.637  f1 0.66


In [4]:


data['matchup'] = data.apply(lambda x: tuple(sorted([x['TEAM_NAME'], x['TEAM_NAME.1']])), axis=1)

# Calculate historical wins and games played between each pair of teams
historical_data = data.groupby(['matchup', 'Season']).apply(lambda x: pd.Series({
    'home_wins': (x['PTS'] > x['PTS.1']).sum(),
    'away_wins': (x['PTS'] < x['PTS.1']).sum(),
    'games_played': len(x)
})).reset_index()

# Calculate win percentage
historical_data['home_win_percentage'] = historical_data['home_wins'] / historical_data['games_played']
historical_data['away_win_percentage'] = historical_data['away_wins'] / historical_data['games_played']

# Merge the win percentage data back into the original dataframe
data = data.merge(historical_data[['matchup', 'home_win_percentage', 'away_win_percentage']], on='matchup', how='left')

data[['TEAM_NAME', 'TEAM_NAME.1', 'home_win_percentage', 'away_win_percentage']]


,TEAM_NAME,TEAM_NAME.1,home_win_percentage,away_win_percentage
0,New York Knicks,Philadelphia 76ers,0.500000,0.500000
1,New York Knicks,Philadelphia 76ers,0.500000,0.500000
2,New York Knicks,Philadelphia 76ers,0.500000,0.500000
3,New York Knicks,Philadelphia 76ers,0.500000,0.500000
4,New York Knicks,Philadelphia 76ers,0.500000,0.500000
...,...,...,...,...
171581,Phoenix Suns,Minnesota Timberwolves,0.500000,0.500000
171582,Phoenix Suns,Minnesota Timberwolves,0.666667,0.333333
171583,Phoenix Suns,Minnesota Timberwolves,0.666667,0.333333
171584,Phoenix Suns,Minnesota Timberwolves,0.750000,0.250000


In [5]:
data

,index,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,Date.1,Home-Team-Win,Days-Rest,Days-Rest.1,Season,Elo_Team,Elo_Team.1,matchup,home_win_percentage,away_win_percentage
0,0,New York Knicks,1.0,1.0,0.0,1.000,48.0,36.0,84.0,0.429,...,2012-11-04,1.0,2.0,4.0,2012,1500.000000,1500.000000,"(New York Knicks, Philadelphia 76ers)",0.500000,0.500000
1,0,New York Knicks,1.0,1.0,0.0,1.000,48.0,36.0,84.0,0.429,...,2012-11-04,1.0,2.0,4.0,2012,1500.000000,1500.000000,"(New York Knicks, Philadelphia 76ers)",0.500000,0.500000
2,0,New York Knicks,1.0,1.0,0.0,1.000,48.0,36.0,84.0,0.429,...,2012-11-04,1.0,2.0,4.0,2012,1500.000000,1500.000000,"(New York Knicks, Philadelphia 76ers)",0.500000,0.500000
3,0,New York Knicks,1.0,1.0,0.0,1.000,48.0,36.0,84.0,0.429,...,2012-11-04,1.0,2.0,4.0,2012,1500.000000,1500.000000,"(New York Knicks, Philadelphia 76ers)",0.500000,0.500000
4,0,New York Knicks,1.0,1.0,0.0,1.000,48.0,36.0,84.0,0.429,...,2012-11-04,1.0,2.0,4.0,2012,1500.000000,1500.000000,"(New York Knicks, Philadelphia 76ers)",0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171581,15114,Phoenix Suns,82.0,49.0,33.0,0.598,48.2,42.5,86.1,0.493,...,2024-04-28,0.0,2.0,2.0,2023,1584.150881,1631.611852,"(Minnesota Timberwolves, Phoenix Suns)",0.500000,0.500000
171582,15114,Phoenix Suns,82.0,49.0,33.0,0.598,48.2,42.5,86.1,0.493,...,2024-04-28,0.0,2.0,2.0,2023,1584.150881,1631.611852,"(Minnesota Timberwolves, Phoenix Suns)",0.666667,0.333333
171583,15114,Phoenix Suns,82.0,49.0,33.0,0.598,48.2,42.5,86.1,0.493,...,2024-04-28,0.0,2.0,2.0,2023,1584.150881,1631.611852,"(Minnesota Timberwolves, Phoenix Suns)",0.666667,0.333333
171584,15114,Phoenix Suns,82.0,49.0,33.0,0.598,48.2,42.5,86.1,0.493,...,2024-04-28,0.0,2.0,2.0,2023,1584.150881,1631.611852,"(Minnesota Timberwolves, Phoenix Suns)",0.750000,0.250000


In [6]:
teams_long = teams_long.sort_values(['TEAM_NAME','Date'])
teams_long

NameError: name 'teams_long' is not defined